In [1]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

import os
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain_community.llms import Ollama
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings

### Define directory with context files

In [2]:
directory = './context/'

In [ ]:
all_documents = []

### Load all files present in given directory

In [3]:
for filename in os.listdir(directory):
    if filename.endswith('.pdf'):
        filepath = os.path.join(directory, filename)
        loader = PyPDFLoader(filepath)
        documents = loader.load()
        all_documents.extend(documents)

### Chunking

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
texts = text_splitter.split_documents(all_documents)

print(f'Total number of chunks: {len(texts)}')

### Load Embedding Model

In [ ]:
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
print('embedding model loaded')

### Load the Vector DB or Create one

In [ ]:
try:
    print('Loading from saved db')
    docsearch = Chroma(embedding_function=embeddings, persist_directory="chroma_db")
    print('document loaded')
except:
    print('Creating new db')
    docsearch = Chroma.from_documents(texts, embeddings, persist_directory="chroma_db")
    docsearch.persist()
    print('document ingested')

### Load a LLM/SLM

In [5]:
llm = Ollama(model="phi3.5")

### Create a prompt

In [6]:
prompt_template = """
You are a legal expert. Your role is to provide clear and accurate legal advice only based on the information from the provided documents and previous conversations with the user. If you don't know the answer, just say that you don't know, definately do not try to make up an answer.

Previous conversations:
{history}

Document context:
{context}

Question: {question}
"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["history", "context", "question"]
)

### Define a function with Retrival Model

In [7]:
def fnAsk():
    qa = RetrievalQA.from_chain_type(llm=llm,
                                    chain_type="stuff",
                                    retriever=docsearch.as_retriever(),
                                    verbose=False,
                                    chain_type_kwargs={
                                        "verbose": False,
                                        "prompt": PROMPT,
                                        "memory": ConversationBufferMemory(
                                            memory_key="history",
                                            input_key="question"),
                                    }, 
                                    return_source_documents=False)

    while True:
        query = input("Question: ")

        print("User: ", query, "\n")
        if query.lower() in ["quit","exit","bye"]:
            print("Bot: Goodbye!")
            break

        result = qa.invoke(query)

        print("Bot: ", result["result"], "\n\n")

### Run the function

In [ ]:
fnAsk()